<a href="https://colab.research.google.com/github/jpsiegel/Projects/blob/master/Tarea2_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Tarea 2 Jan P. Siegel - Deep Learning

In [1]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU

  Created wheel for gputil: filename=GPUtil-1.4.0-cp37-none-any.whl size=7411 sha256=60c8bf61fb0ead4fb990a3df5d7f9d6969cc5ebd0d7c1777d99e83ac4b430798
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil


In [2]:
GPUs = GPU.getGPUs()
gpu = GPUs[0]  # Only one GPU on Colab and not guaranteed

def printm():
  process = psutil.Process(os.getpid())
  print("RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " | Used: " + humanize.naturalsize(process.memory_info().rss))
  print("VRAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

printm()

RAM Free: 12.8 GB  | Used: 117.8 MB
VRAM Free: 15109MB | Used: 0MB | Util   0% Total 15109MB


##Parte 1


###Actividad 1

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

sms_data_path = '/content/drive/MyDrive/DL/T2/SMSSpamCollection'

Mounted at /content/drive


In [ ]:
!pip install d2l

In [76]:
# parse data set

from string import punctuation
from d2l import torch as d2l

#  Reads sms data file
raw_data = open(sms_data_path, "r")
strings = raw_data.readlines()

vocab_freq = {} # word : count

data = []
labels = []
for s in strings:
  s = s.strip("\n")
  listed = s.split("\t")
  label = 1 if listed[0] == "spam" else 0 # 1 is spam and 0 is not spam
  phrase = listed[1].lower()
  phrase = "".join([c for c in phrase if c not in punctuation]) # eliminate punctuation
  words = phrase.split(" ")
  for word in words:
    try:
      vocab_freq[word] += 1
    except KeyError:
      vocab_freq[word] = 1
  data.append(words)
  labels.append(label)

vocab = list(vocab_freq.keys())
list_len = [len(i) for i in data]
longest_phrase = max(list_len)
print("Data:", data[:3]) # lists_of_words
print("Labels:", labels) # list of corresponding labels
print("Vocab length:", len(vocab), "different words") # vocabulary of 9662 words
print("Longest phrase:", longest_phrase, "words")
print("Total data:", len(data), "labeled phrases") # spamm: 747, no spamm: 4827


Data: [['go', 'until', 'jurong', 'point', 'crazy', 'available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', 'cine', 'there', 'got', 'amore', 'wat'], ['ok', 'lar', 'joking', 'wif', 'u', 'oni'], ['free', 'entry', 'in', '2', 'a', 'wkly', 'comp', 'to', 'win', 'fa', 'cup', 'final', 'tkts', '21st', 'may', '2005', 'text', 'fa', 'to', '87121', 'to', 'receive', 'entry', 'questionstd', 'txt', 'ratetcs', 'apply', '08452810075over18s']]
Labels: [0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [68]:
# obtain GloVe word embedding

embedding_dim = "100" # dimensiones aceptadas 50, 100, 200, 300 (act 2 y 5)
glove_embedding = d2l.TokenEmbedding(f'glove.6b.{embedding_dim}d') 

In [94]:
from torch.nn.utils.rnn import pad_sequence

embeds = glove_embedding[vocab] # generate embedding for every word in vocabulary
print("Embedding shape:", embeds.shape) # we have 9662 vectors of 100 dims
word_idx = vocab.index("go")
word_tensor = embeds[word_idx]

# associate word with glove embedding
embedding_dict = {} # { word: embedding_vector }
for i in range(len(vocab)):
  embedding_dict[list(vocab)[i]] = embeds[i]

# parse from list of words to tensor for each phrase, padding included
processed_data = []
for p in range(len(data)): # phrase_idx
  word_tensors = []
  for w in range(len(data[p])): # word_idx
    current_word = data[p][w]
    word_idx = vocab.index(current_word)
    word_tensor = embeds[word_idx] # word to tensor according to glove embedding
    word_tensors.append(word_tensor)
  # pasar a 1 tensor para toda la frase de dim largo_phrase x 100
  phrase_tensor = torch.stack(word_tensors) # de [tensor.shape(1,100),tensor.shape(1,100)] a tensor.shape(2,1,100)
  processed_data.append(phrase_tensor)
  # con esto tengo un tensor(x,100) para cada frase, donde x es la cantidad de palabras en esa frase
# finalmente dejamos todos los tensores del largo de la frase mas larga, paddeados con 0s
padded_data = pad_sequence(processed_data, batch_first=True)

print(data[:3]) # primera frase tiene 20 elems, padded deberia quedar con 171 que es frase mas larga
print(processed_data[0].shape)
print(padded_data[0].shape)
print(padded_data[0])

Embedding shape: torch.Size([9662, 100])
[['go', 'until', 'jurong', 'point', 'crazy', 'available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', 'cine', 'there', 'got', 'amore', 'wat'], ['ok', 'lar', 'joking', 'wif', 'u', 'oni'], ['free', 'entry', 'in', '2', 'a', 'wkly', 'comp', 'to', 'win', 'fa', 'cup', 'final', 'tkts', '21st', 'may', '2005', 'text', 'fa', 'to', '87121', 'to', 'receive', 'entry', 'questionstd', 'txt', 'ratetcs', 'apply', '08452810075over18s']]
torch.Size([20, 100])
torch.Size([171, 100])
tensor([[-0.0789,  0.4616,  0.5778,  ...,  0.2635,  0.5940,  0.2674],
        [-0.1100, -0.1945, -0.0746,  ...,  0.6600, -0.0406, -0.2722],
        [-0.1955,  0.3734,  0.4894,  ...,  0.1966,  0.9504, -0.6675],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])


In [96]:
# split data set 

split_frac = 0.8 # 80% train, 10% validation, 10% test
amount = len(padded_data)

train_x = padded_data[0:int(split_frac*amount)]
train_y = labels[0:int(split_frac*amount)]
remaining_x = padded_data[int(split_frac*amount):]
remaining_y = labels[int(split_frac*amount):]
valid_x = remaining_x[0:int(len(remaining_x)*0.5)]
valid_y = remaining_y[0:int(len(remaining_y)*0.5)]
test_x = remaining_x[int(len(remaining_x)*0.5):]
test_y = remaining_y[int(len(remaining_y)*0.5):]

print(len(train_x))
print(len(valid_x))
print(len(test_x))

4459
557
558


La idea de implementar un embedding para las palabras es poder pasar de datos en texto a tensores vectoriales numéricos, esto permite expresar las palabras en un formato que les añada riqueza en su descripción semántica (haciendo que palabras similares tengan valores cercanos) y además las haga consumibles por un modelo de aprendizaje automático.

En particular, GloVe no captura solamente las características estadísticas locales como Word2Vec, sino también las (Glo)bales para crear el (Ve)ctor deseado. Para lograr esto,  toma en cuenta relaciones semánticas locales, pero también deriva relaciones semánticas a partir de una matriz de co-ocurrencia, donde se indica qué tan probable es que una palabra aparezca en el contexto de otra. La gracia de GloVe es que puede expresar esta relación en un vector limpiamente con escasa pérdida de información.

###Actividad 2

In [98]:
# OJOO Con Softmax y CrossEntropy la red no aprende nada, pero cuando pongo LogSoftMax con NLLoss la red empieza a aprender 

import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence

class RNN(nn.Module):
  
  def __init__(self, input_dim, hidden_dim, num_layers, embedding_dict):
    super().__init__() # input_dim es la dimension del x_t, es la dimension que te deja el embedding (glove)
    self.hidden_dim = hidden_dim # dimensiones de h_t (capa oculta)
    self.num_layers = num_layers # cantidad de capas (filas) ocultas
    self.embedding_dict = embedding_dict # { word: embedding_vector }

    self.rnn = nn.RNN(input_dim, hidden_dim, num_layers, batch_first=True)
    self.fc1 = nn.Linear(hidden_dim, 60) # capa intermedia, de ht a st
    self.activation = nn.ReLU()
    self.linear_out = nn.Linear(60, 2) # de st a yt
    self.softmax = nn.Softmax(2)  

  # This method defines the forward pass of the RNN
  def forward(self, input):
    batch_size, _ = input.size()

    # Initializing hidden state for first input
    h0 = self.init_hidden(batch_size)
    # Passing in the input and hidden state to obtain output
    _, hidden_state = self.rnn(input.unsqueeze(2), h0)
    out = self.linear_out(hidden_state.squeeze())
    return out
  
  def forward(self, text, text_lengths):
      
    #text = [batch size,sent_length]
    embedded = self.embedding(text)
    #embedded = [batch size, sent_len, emb dim]
      
    #packed sequence
    packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths,batch_first=True)
        
    packed_output, (hidden, cell) = self.lstm(packed_embedded)
    #hidden = [batch size, num layers * num directions,hid dim]
    #cell = [batch size, num layers * num directions,hid dim]
        
    #concat the final forward and backward hidden state
    hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
                
    #hidden = [batch size, hid dim * num directions]
    dense_outputs=self.fc(hidden)

    #Final activation function
    outputs=self.act(dense_outputs)
        
    return outputs
    
  # This method generates the first hidden state of zeros for the forward pass
  # This creates a tensor of zeros in the shape of our hidden states.
  def init_hidden(self, batch_size):
    hidden = torch.zeros(self.num_layers, batch_size, self.hidden_dim)
    return hidden

rnn_model = RNN(input_dim=100, hidden_dim=80, num_layers=1, embedding_dict=embedding_dict)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
rnn_model.to(device)
print(rnn_model)

# No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(rnn_model):,} trainable parameters')

RNN(
  (rnn): RNN(100, 80, batch_first=True)
  (fc1): Linear(in_features=80, out_features=60, bias=True)
  (activation): ReLU()
  (linear_out): Linear(in_features=60, out_features=2, bias=True)
  (softmax): Softmax(dim=2)
)
The model has 19,542 trainable parameters


El número de parametros depende indirectamente del dataset utilizado, si usamos un dataset de texto muy grande y complejo, necesitaremos más dimensiones para obtener un buen embedding, lo que ser reflejará en la cantidad de parámetros de Wxh. Sin embargo, si se mantienen las dimensiones del embedding, la cantidad de parámetros no se verá afectada al cambiar de dataset.

Entonces, podemos ver que se puede reducir la cantidad de parámetros de la red cambiando el embedding usado, por ejemplo si usamos GloVe con 50d (en vez de 100d), tendremos un tensor de 50 dimensiones para capturar las features de cada palabra, en vez de 100 dimensiones para cada palabra. Esto reducirá la cantidad de parámetros entrenables en W_xh, todo lo anterior sin alterar la dimensionalidad de la capa oculta h_t.


In [103]:
new_rnn_model = RNN(input_dim=100, hidden_dim=120, num_layers=1, embedding_dict=embedding_dict)
print(f'The new model has {count_parameters(new_rnn_model):,} trainable parameters')


The new model has 34,022 trainable parameters


Al pasar de 80 a 120 en la dimension de la capa oculta h_t, vemos que la cantidad de parámetros pasa de 19542 a 34022, es decir, un aumento del 74% lo que es entendible puesto que un aumento en h_t implica incrementar la cantidad de parámetros para todas las transformaciones del modelo, es decir, para W_hx, W_hh y W_yh

##Referencias
https://www.kaggle.com/anindya2906/glove6b